Cartar de posible desvio para la union

In [2]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

In [6]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,799.62,81.377149
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,1605.19,99.368574
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,6454.93,122.054586


In [7]:
xlsx_avance.head(3)

,COD COS,INST,COD CA,NOMBRE,COMPROMISO,ENTREGAS,%ENTREGA,TOTAL AREA,COSECHADO,SIN COSECHAR,...,TCH,%DIFF AVANCE,TCH EST,PRODUC EST,EST PRODUC COSECHADO,DIF. EST ENTREGA,%DIF. EST ENTREGA,TN EN CAMPO,COMPROMISO RES,DIFERENCIA
0,1,50,515,AGUILERA OLGA RIVERO VDA DE,30572.11,31123.38,1.018032,618.28,545.74,66.57,...,57.029684,0.135357,57.639599,35292.15,31964.10,-840.72,1.027012,3328.05,0.00,3328.05
1,2,69,1180,BALCAZAR BALCAZAR JESUS E.,10965.29,11973.82,1.091975,200.37,198.31,NaN,...,60.379305,0.102256,53.046741,10531.90,10531.90,1441.92,0.879577,0.00,0.00,0.00
2,3,69,911,BALCAZAR JUSTINIANO MARIO,5422.67,4542.55,0.837696,122.96,94.11,28.20,...,48.268516,0.072325,51.428688,6288.70,4971.95,-429.40,1.094528,1316.75,880.12,436.63


In [12]:
fecha_inf = '26 de noviembre de 2025'
fecha_form = '26-11-2025'

fecha_img = ' de noviembre de 2025'

cod_grupos = [
    12,
39,
59,
62,
71,
88,
259,
279,
282,
286,
321,
324,
352,
363,
377,
382,
390,
459,
475,
503,
513,
568,
606,
614,
628
]


In [15]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHADO'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posible_desvio.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/_' + str(avance['COD CA'].iloc[0]) + '_RPDC_' + fecha_form + '_' + avance['NOMBRE'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE'].iloc[0])

In [16]:
crear_reporte(cod_grupos)

Se generó reporte de grupo: 12  -  SAUCEDO CERRUTI SONIA
Se generó reporte de grupo: 39  -  MONASTERIO FOIANINI SYLVIA D.
Se generó reporte de grupo: 59  -  ENRIQUEZ ROJAS BENEDICTO
Se generó reporte de grupo: 62  -  BARBOZA VACA WILLY
Se generó reporte de grupo: 71  -  ESPINOZA ESTHER AGUILAR DE
Se generó reporte de grupo: 88  -  FLORES CONDORI MARITZA
Se generó reporte de grupo: 259  -  MELGAREJO CRESPO RENE
Se generó reporte de grupo: 279  -  VASQUEZ SOLIZ SANTOS
Se generó reporte de grupo: 282  -  SALAZAR ARNEZ LIDER
Se generó reporte de grupo: 286  -  LAVERAN VACA ZENON
Se generó reporte de grupo: 321  -  OLGUIN GUTIERREZ NAIR ROLANDO
Se generó reporte de grupo: 324  -  SALAZAR ARNEZ HECTOR
Se generó reporte de grupo: 352  -  ANCE CONDORI JUANITO
Se generó reporte de grupo: 363  -  CHAVEZ RUTH TRUJILLO DE
Se generó reporte de grupo: 377  -  RAMOS SORUCO GREGORIO
Se generó reporte de grupo: 382  -  TAPIA MARTINEZ FRANCISCO
Se generó reporte de grupo: 390  -  RICALDI SUAREZ MAXIMO
S